<a href="https://colab.research.google.com/gist/resmaili/c88f7cf2bfd97e6ecce46d791dc5df58/agu_2024_access_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial: Visualizing NOAA satellite data from the cloud**
Author: Rebekah Esmaili, PhD

This tutorial will show the basic steps to accessing GOES-16 ABI data on Amazon Web Services (AWS) and make an [Sandwich RGB](https://www.star.nesdis.noaa.gov/goes/documents/SandwichProduct.pdf) of Hurricane Rafael from November 2024!

The main steps will be:
1. Import relevant packages
2. Search cloud repositories for NOAA satellite data
3. Import the data into memory
4. Make a plot of a single GOES-16 ABI channel
4. Reformat data for a combined imagery plot
5. Plot imagery on a map

This is a beginner-intermediate level tutorial and will take 30-45 minutes to complete Sections 1-4. Section 5 may take longer depending on your Python and GIS experience. Special thanks to the [CIMSS Satellite Blog](https://cimss.ssec.wisc.edu/satellite-blog/archives/61515) for documenting this example.

## 1 Import Relevant Packages
**bold text**


We will use three packages for this project (`matplotlib`, `xarray`, `numpy`, and `s3fs`). We have to install s3fs, which we can do using pip. This may take ~20s to complete.

In [ ]:
!pip install s3fs cartopy --quiet

Next, let's import the packages.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import xarray as xr
import s3fs
import numpy as np
import cartopy.crs as ccrs

The package s3fs is file interface for Amazon S3 (Simple Storage Service) buckets, so you can browse and search for data. NOAA's Open Data Dissemination (NODD) program is increasing access to satellite data, including GOES and JPSS.

## 2 Search cloud repositories for NOAA satellite data


In this tutoral, we'll look at [GOES-16 Mesoscale data in S3](https://registry.opendata.aws/noaa-goes/). Mesoscale files begin with `ABI-L1b-RadM`. Full-disk and CONUS data have greater spatial coverage and also are available on the cloud. You can visually browse these datasets via the provided link and find "browse bucket" underneath the GOES satellite of interest. You can also brose [JPSS datasets in S3](https://registry.opendata.aws/noaa-jpss/).

First, we have to initialize the filesystem. We pass in the keyword `anon` because we are not going to pass any login information since the data are public.

In [ ]:
fs = s3fs.S3FileSystem(anon=True)

You can manually browse the contents of the S3 bucket using the link above, or you can search in the command line. The S3 bucket is named `noaa-goes16`, and using the `s3fs` ls command (we'll only print the first 3 entries for brevity):

In [ ]:
fs.ls('noaa-goes16')[0:3]

We can repeat the search above by extending the argument from `noaa-goes16` to `noaa-goes16/ABI-L1b-RadM` and running the command again. To save some time, let's construct the full path below:

In [ ]:
bucket_name = 'noaa-goes16'
product_name = 'ABI-L1b-RadM'
year = 2024
doy = 309
hour = 18

path = bucket_name + '/' + product_name + '/' + str(year) + '/' + str(doy).zfill(3) + '/' + str(hour).zfill(2) + '/'
print(path)

Since we're looking at the mesosector scan, this search will still yield a lot of results:

In [ ]:
files = fs.ls(path)
print(files[0:5])

If you've seen satellite data files before, you'll know they have long filenames. Let's break it down:

`[sensor and product name]-[scan sector]_[satellite]_s[scan start time]_e[scan end time]_c[creation time].nc`

To save a little bit of time, let's cheat a bit and only work with the first available time stamp (`'20243091800'`). The sandwich product takes the difference of ABI channels 3 and 13. The snippet of code below will search all the files and only keep the ones that have `'C03'` or `'C13'` and the start time `'202430918002'`.

I'm using a list comprehension, which is a shorter way of writing a loop. For reference, the original loop is commented out:

In [ ]:
# Long form of writing a loop
# for file in files:
#     if ('C03_G16_s2024309180' in file) | ('C13_G16_s2024309180' in file):
#         print(file)

matches = [file for file in files if ('C03_G16_s202430918002' in file) | ('C13_G16_s202430918002' in file)]
print(matches)

Now that we have found our files in the satellite data "haystack" we can finally open them:

In [ ]:
remote_obj_ch3 = fs.open(matches[0], mode='rb')
remote_obj_ch13 = fs.open(matches[1], mode='rb')

## 3 Import the data into memory

We will need to use `xarray` to read*/*interpret the netCDF file.

In [ ]:
abi_ch3 = xr.open_dataset(remote_obj_ch3, engine='h5netcdf')
abi_ch13 = xr.open_dataset(remote_obj_ch13, engine='h5netcdf')

We can print the header information to see the file contents:

In [ ]:
abi_ch3

## 4 Make a plot of a single GOES-16 ABI channel

We can learn a lot about the file from the header. For example, the file dimensions are 1000 x 1000, there's a useful variable named `Rad` (for Radiance), and the `x` and `y` coordinates in the geostationary projection.


Let's preview what these two images look like using `matplotlib` and `imshow`. The title and labels are optional. We set `origin` to upper to prevent the images from being flipped.

In [ ]:
plt.figure(figsize=[10,10])
plt.title("GOES-16 ABI Channel 3 (0.86 μm)")
tmp = plt.imshow(abi_ch3.Rad, cmap='Greys_r', origin='upper')
plt.colorbar(tmp, label=("Brightness Temperature (K)"))
plt.show()

In [ ]:
plt.figure(figsize=[10,10])
tmp = plt.imshow(abi_ch13.Rad, cmap='Greys', origin='upper')
plt.title("GOES-16 ABI Channel 13 (10.4 μm)")
plt.colorbar(tmp, label=("Brightness Temperature (K)"))
plt.show()

## 5 Reformat data for a combined imagery plot

Our goal is to create a sandwich product, which combines ABI channel 3 and channel 13 radiances. First, let's extract the radiances from the file:

In [ ]:
C13 = abi_ch13.Rad.values
C03 = abi_ch3.Rad.values

We need to make sure both are the same resolution, but unfortunately they are not. Channel 3 is 1 km and Channel 13 is 2 km. You can use the shape command to check the array size:

In [ ]:
C13.shape, C03.shape

There are many techniques to change the data reoslution, but a fast and simple way to change the gridding to skip every other pixel. Python can skip indices using the double colons (::) followed by an integer. We'll overwrite the original variable with this new smaller resolution variable.

In [ ]:
C03 = C03[::2, ::2]
C03.shape, abi_ch13.Rad.shape

We want to overlay the two images. You can change the values below, but we'll set the opacity to 0 (fully transparent) in warm regions so that we can only see the coldest cloud tops on Channel 13, but use Channel 3 to show the surface.

In [ ]:
mask = (abi_ch13.Rad.values >= 60)
alpha = np.where(mask, 0.0, 0.9)
alpha

Finally, we'll make a plot of our hard work!

In [ ]:
plt.figure(figsize=[10,10])
plt.imshow(abi_ch3.Rad, cmap='Greys_r', alpha=0.8, origin='upper')
tmp = plt.imshow(abi_ch13.Rad.values, cmap='jet_r', alpha=alpha, vmax=60, origin='upper')
plt.colorbar(tmp)
plt.show()

## 6 Plot imagery on a map (advanced)

This next section is advanced, so if you are new and find it confused, that's okay! It can be a useful template for plotting other GOES imegery and products.

The GOES imager projection is the gridding system used by all GOES ABI imagery channels and many products. It is based on the viewing projection of the satellite. To learn more, you can [read this excellent guide](https://www.star.nesdis.noaa.gov/atmospheric-composition-training/satellite_data_goes_imager_projection.php) written by Dr. Amy Huff.

We'll need to read several viewing (`sat height` and `central longitude`) and earth geometry variables (`semi_major` and `semi_minor`) from the ABI file. Since both channel 3 and 13 are from the same instrument on the same satellite, the geometries will be the same for both. I'll extract these values from the `goes_imager_projection` variable in the Channel 13 file (`abi_ch13`).

In [ ]:
geom = abi_ch13.goes_imager_projection

sat_height = geom.perspective_point_height
central_lon = geom.longitude_of_projection_origin
semi_major = geom.semi_major_axis
semi_minor = geom.semi_minor_axis

Next, we'll need to define three objects in [Cartopy](https://scitools.org.uk/cartopy/docs/latest/) to define the coordinate reference system (crs) using `ccrs.Geostationary` before we can make a map. We start by defining the `Globe` parameter of the crs using Earth's `semi_major` and `semi_minor` axes.

Then we define define the crs using the `central_lon`, `sat_height`, and `globe` variables.

In [ ]:
globe = ccrs.Globe(semimajor_axis=semi_major, semiminor_axis=semi_minor)
crs = ccrs.Geostationary(central_longitude=central_lon, satellite_height=sat_height, globe=globe)

That's all we need for the map!

We also need to define the edges of the image. We can do this by multiplying the min and max values of the

In [ ]:
X = abi_ch13['x'][:] * sat_height
Y = abi_ch13['y'][:] * sat_height

imgExtent = (X.min(), X.max(), Y.min(), Y.max())

We can re-use a lot of the code from the plot we made in Section 5, but we need to add the `extent` and set it to the boundaries of the image.

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(projection=crs)
ax.coastlines('10m', color='white', linewidth=1.5)
ax.imshow(abi_ch3.Rad, origin='upper', cmap='Greys_r', alpha=0.8, extent=imgExtent)
temp = ax.imshow(abi_ch13.Rad.values, origin='upper', cmap='jet_r', extent=imgExtent, alpha=alpha, vmax=60)
plt.show()

Now we can see the coastlines!

# Summary

In this tutorial, we covered steps to access and display NOAA satellite datasets. For more information on the datasets that are available on the cloud, visit the [NOAA Open Data Dissemination (NODD)](https://www.noaa.gov/information-technology/open-data-dissemination) website. To learn more about NOAA satellite data products, visit [NOAA OneStop](https://data.noaa.gov/onestop/).